In [1]:
from sklearn.metrics import log_loss, accuracy_score

In [5]:
%run validation.ipynb

### Mean Decrease Impurity

In [ ]:
def feature_importance_mdi(fit, feature_names):
    """
    
    feature importance based on IS mean impurity reduction
    
    """
    
    df0 = {i:tree.feature_importances_ for i, tree in enumerate(fit.estimateors_)}
    df0 = pd.DataFrame.from_dict(df0, orient = 'index')
    df0.columns = feature_names
    df0 = df0.replace(0, np.nan) # because max_features = 1
    imp = pd.concat({'mean':df0.mean(), 'std':df0.std()*df0.shape[0]**-0.5}, axis=1)
    imp /= imp['mean'].sum()
    
    return imp

### Mean Decrease Accuracy

In [ ]:
def feature_importance_mda(clf, X, y, cv, sample_weight, t1, pct_embargo, scoring='neg_log_loss'):
    """
    
    feature importance based on OOS score reduction
    
    """
    
    if scoring not in ['neg_log_loss', 'accuracy']:
        raise Exception ('wrong scoring method')
    
    cv_gen = PurgedKFold(n_splits=cv, t1=t1, pct_embargo=pct_embargo) # purged cv
    scr_0, scr_1 = pd.Series(), pd.DataFrame(columns=X.columns)
    for i, (train, test) in enumerate(cv_gen.split(X=X))
        X0, y0, w0 = X.iloc[train, :], y.iloc[train], sample_eights.iloc[train]
        X1, y1, w1 = X.iloc[test, :], y.iloc[test], sample_weights.iloc[test]
        fit = clf.fit(X=X0, y= y0, sample_weight=w0.values)
        if scoring=='neg_log_loss':
            prob = fit.predict_probs(X1)
            scr0.loc[i]=-log_loss(y1, prob, sample_weight=w1.values, labels=clr.classes_)
        else:
            pred = fit.predict(X1)
            scr0.loc[i]=accuracy_score(y1, pred, sample_weight=w1.values)
        for j in X.columns:
            X1_ = X1.copy(deep=True)
            np.random.suffle(X1_[j].values) # permuation of a single column
            if scoring=='neg_log_loss':
                prob = fit.predict_probs(X1_)
                scr1.loc[i,j]=-log_loss(y1, prob, sample_weight=w1.values, labels=clr.classes_)
            else:
                pred = fit.predict(X1_)
                scr0.loc[i,j]=accuracy_score(y1, pred, sample_weight=w1.values)
    imp = (-src1).add(scr0, axis=0)
    if scoring == 'neg_log_loss': imp = imp/-src1
    else: imp = imp/(1.0-src1)
    imp = pd.concat({'mean':imp.mean(), 'std':imp.std()*imp.shape[0]**-0.5}, axis=1)